In [ ]:
# Provide the transformation code that:
# o Cleans null values
# o Validates column types
# o Aggregates daily transaction totals by bank_id

def transform(filepath: str) -> str:
    """
    Clean, validate, and aggregate transaction data.
    Removes anomalous amounts (> 10000).
    Saves to a transformed CSV file (ready for MySQL load).
    """
    import pandas as pd

    # Step 0: Load CSV
    df = pd.read_csv(filepath)

    # Step 1: Drop nulls
    df = df.dropna()

    # Step 2: Enforce column types
    df["bank_id"] = df["bank_id"].astype(str)
    df["customer_id"] = df["customer_id"].astype(str)
    df["amount"] = df["amount"].astype(float)
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    # Step 3: Aggregate daily totals per bank
    df["date"] = df["timestamp"].dt.date
    agg = (
        df.groupby(["bank_id", "date"], as_index=False)
        .agg(daily_total_by_bank=("amount", "sum"))
    )

    # Step 4: Save transformed file
    outpath = filepath.replace(".csv", "_transformed.csv")
    agg.to_csv(outpath, index=False)

    print(f"Transformed CSV written to {outpath}")
    print(agg.head())  # preview first rows
    return outpath
